<a href="https://colab.research.google.com/github/karentlugo-kavak/Filtros_masivo_indivisuales/blob/main/orig_Filtros_Individual_Masivo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#ORIG


In [2]:
from __future__ import annotations
from datetime import datetime, timedelta
import os
import warnings
import pandas as pd
import csv
import unicodedata
import re

warnings.filterwarnings("ignore")
pd.options.mode.chained_assignment = None

# ==== Pandas ====
try:
    pd.set_option("mode.copy_on_write", True)
    pd.options.mode.string_storage = "pyarrow"
except Exception:
    pass
try:
    import pyarrow
    _PYARROW_OK = True
except Exception:
    _PYARROW_OK = False

# ===================== entorno =====================
try:
    import google.colab
    _COLAB = True
except Exception:
    _COLAB = False

if _COLAB:
    from google.colab import drive, files
    drive.mount('/content/drive', force_remount=False)

# ===================== fechas y utilidades =====================
MESES_ES = {
    1: 'Enero', 2: 'Febrero', 3: 'Marzo', 4: 'Abril', 5: 'Mayo', 6: 'Junio',
    7: 'Julio', 8: 'Agosto', 9: 'Septiembre', 10: 'Octubre', 11: 'Noviembre', 12: 'Diciembre'}

def hoy_yyyymmdd() -> str:
    return datetime.now().strftime('%Y%m%d')

BASE = '/content/drive/Shared drives/Archivos_BO' if _COLAB else 'Shared drives/Archivos_BO'

def construir_ruta_dia(carpeta_rel: str, fecha: datetime) -> str:
    return os.path.join(BASE, carpeta_rel, str(fecha.year), MESES_ES[fecha.month])

def nombre_objetivo(base_nombre: str, fecha: datetime) -> str:
    return f"{base_nombre}_{fecha.strftime('%Y%m%d')}"

# ============================ catálogo de archivos ============================
TIPOS = {
    # -------------------------- Administración ---------------------------
    'CDD': {
        'carpeta': 'Reportes_Consolidados/CDD',
        'documento': 'CDD_ACUMULADO',
        'columnas': [
            "Número_del_deudor","Nombre_del_Deudor","Número_de_Contrato","Fecha_de_cesión",
            "Fideicomisario","Fideicomitente","Fecha_Archivo","Nombre_Archivo"
        ],
    },
    'COB': {
        'carpeta': 'Reportes_Consolidados/COB',
        'documento': 'COB_ACUMULADO',
        'columnas': [
            "Número_de_recibo","Número_de_deudor","Número_de_contrato","Concepto_a_ceder",
            "Fecha_de_depósito","Fecha_de_aplicación","Monto_de_distribución","Fideicomisario",
            "Factura_de_Cliente","Pago","Contrato_Credito","Fideicomitente","Fecha_Archivo","Nombre_Archivo"
        ],
    },
    'CRB': {
        'carpeta': 'Reportes_Consolidados/CRB',
        'documento': 'CRB_ACUMULADO',
        'columnas': [
            "Número_del_deudor","Número_de_recibo","Fecha_de_recibo","Monto_del_recibo",
            "Moneda","Banco","Cuenta_bancaria","Referencia_bancaria","Status",
            "Fecha_de_reversión","Código_de_reversión","Comentario_a_la_reversión",
            "Fideicomitente","Fecha_Archivo","Nombre_Archivo"
        ],
    },
    'NCR': {
        'carpeta': 'Reportes_Consolidados/NCR',
        'documento': 'NCR_ACUMULADO',
        'columnas': [
            "Numero de contrato","Tipo","Monto","Fecha Aplicación","Tipo Aplicación",
            "Codigo Aplicación","Fecha_Archivo","Nombre_Archivo"
        ],
    },
    'DELETION_CRB': {
        'carpeta': 'Reportes_Consolidados/DELETION_CRB',
        'documento': 'DELETION_CRB_ACUMULADO',
        'columnas': [
            "Número_del_deudor","Número_de_recibo","Fecha_de_recibo","Monto_del_recibo",
            "Moneda","Banco","Cuenta_bancaria","Referencia_bancaria","Status",
            "Fecha_de_reversión","Código_de_reversión","Comentario_a_la_reversión",
            "Fideicomitente","Fecha_Archivo","Nombre_Archivo"
        ],
    },
    'DELETION_COB': {
        'carpeta': 'Reportes_Consolidados/DELETION_COB',
        'documento': 'DELETION_COB_ACUMULADO',
        'columnas': [
            "Número_de_recibo","Número_de_deudor","Número_de_contrato","Concepto_a_ceder",
            "Fecha_de_depósito","Fecha_de_aplicación","Monto_de_distribución","Fideicomisario",
            "Factura_de_Cliente","Pago","Contrato_Credito","Fideicomitente","Fecha_Archivo","Nombre_Archivo"
        ],
    },

    # ------------------------------- CxC ---------------------------------
    'APLICACIONES': {
        'carpeta': 'APLICACIONES',
        'documento': 'estatus_aplicaciones',
        'columnas': [
            "fecha_generacion","id_fideicomiso","alias","numero_recibo","monto","monto_fondeado",
            "fecha_distribucion","clave_deudor","nombre_deudor","esta_conciliada","fecha_deposito",
            "no_cuenta","abreviacion","id_aplicacion","monto_transferencia","numero_contrato",
            "concepto_archivo","fideicomisario","identificador_cliente","fecha_archivo","fecha_reversion",
            "nombre_archivo","fideicomitente"
        ],
    },
    'RECIBOS': {
        'carpeta': 'RECIBOS',
        'documento': 'estatus_recibos',
        'columnas': [
            "fecha_generacion","id_fideicomiso","alias","banco","numero_cuenta","numero_recibo",
            "fecha_deposito","monto","referencia_bancaria","conciliado","saldo_restante","id_recibo",
            "clave_deudor","nombre_deudor","nombre_archivo","fideicomitente"
        ],
    },
    'DEPOSITOS': {
        'carpeta': 'DEPOSITOS',
        'documento': 'estatus_depositos',
        'columnas': [
            "fecha_generacion","id_fideicomiso","alias","banco","numero_cuenta","categoria",
            "fecha_deposito","monto_deposito","referencia","concepto","referencia_ampliada",
            "saldo_restante","esta_conciliado","id_deposito"
        ],
    },
    'TRANSFERENCIAS': {
        'carpeta': 'TRANSFERENCIAS',
        'documento': 'estatus_transferencias',
        'columnas': [
            "fecha_generacion","id_deposito","id_recibo","monto_transferencia","fecha_transferencia",
            "id_tipo_transferencia","nombre_regla","descripcion"
        ],
    },

    # ------------------------------- Errores ------------------------------
    'ERRORES_CDD': {
        'carpeta': 'ERRORES/Reportes_Consolidados/CDD',
        'documento': 'ErroresCDD',
        'columnas': [
            "ID","Key","Value","line","Id_nom","Nombre","Recibo","Fecha","Fidecomisario",
            "Fidecomitente","Fecha_Archivo","Nombre_Archivo"
        ],
    },
    'ERRORES_COB': {
        'carpeta': 'ERRORES/Reportes_Consolidados/COB',
        'documento': 'ErroresCOB',
        'columnas': [
            "ID","Key","Value","line","Numero_Recibos","Clave_Deudor","Contrato","Concepto_A_ceder",
            "Fecha_Deposito","Fecha_Aplicacion","Monto_Distribuido","Fideicomiso","Fecha_Archivo","Nombre_Archivo"
        ],
    },
    'ERRORES_CRB': {
        'carpeta': 'ERRORES/Reportes_Consolidados/CRB',
        'documento': 'ErroresCRB',
        'columnas': [
            "ID","Key","Value","line","Deudor","Numero_Recibo","Fecha_Deposito","Monto","Moneda",
            "Fideicomiso","Cuenta","Referencia","Fideicomitente","Fecha_Archivo","Nombre_Archivo"
        ],
    },
    'ERRORES_DELETIONCOB': {
        'carpeta': 'ERRORES/Reportes_Consolidados/DELETION_COB',
        'documento': 'ErroresDELETIONCOB',
        'columnas': [
            "ID","Key","Value","line","Numero_Recibos","Clave_Deudor","Contrato","Concepto_A_ceder",
            "Fecha_Deposito","Fecha_Aplicacion","Monto_Distribuido","Fideicomiso","Fecha_Archivo","Nombre_Archivo"
        ],
    },
    'ERRORES_NCR': {
        'carpeta': 'ERRORES/Reportes_Consolidados/NCR',
        'documento': 'ErroresNCR',
        'columnas': [
            "id","numero_contrato","tipo_liberacion","monto","fecha_aplicacion","tipo_aplicacion",
            "codigo_aplicacion","id_tiempo","id_archivo","created_at","id_estatus",
            "numero_linea_archivo","estatus","Fecha_Archivo","Nombre_Archivo"
        ],
    },
}

# =================== rutas masivas por usuario =================
rutas_usuarios = {
    "JC": f"{BASE}/Por_Usuario/Josue/Valores.xlsx",
    "ED": f"{BASE}/Por_Usuario/Edith/Valores.xlsx",
    "MD": f"{BASE}/Por_Usuario/Mitchell/Valores.xlsx",
    "MS": f"{BASE}/Por_Usuario/Mike/Valores.xlsx",
    "IT": f"{BASE}/Por_Usuario/Itzel/Valores.xlsx",
    "KT": f"{BASE}/Por_Usuario/Karent/Valores.xlsx",
    "ST": f"{BASE}/Por_Usuario/Stefan/Valores.xlsx",
}

# ====== mapeos de columnas  ======
USUARIO_A_DESTINO = {
    # ----------------------- Administración -----------------------
    'CDD': {
        'columna_01': 'Número_del_deudor',
        'columna_02': 'Nombre_del_Deudor',
        'columna_03': 'Número_de_Contrato',
        'columna_04': 'Fecha_de_cesión',
        'columna_05': 'Fideicomisario',
        'columna_06': 'Fideicomitente',
        'columna_07': 'Fecha_Archivo',
        'columna_08': 'Nombre_Archivo',
    },
    'COB': {
        'columna_01': 'Número_de_recibo',
        'columna_02': 'Número_de_deudor',
        'columna_03': 'Número_de_contrato',
        'columna_04': 'Concepto_a_ceder',
        'columna_05': 'Fecha_de_depósito',
        'columna_06': 'Fecha_de_aplicación',
        'columna_07': 'Monto_de_distribución',
        'columna_08': 'Fideicomisario',
        'columna_09': 'Factura_de_Cliente',
        'columna_10': 'Pago',
        'columna_11': 'Contrato_Credito',
        'columna_12': 'Fideicomitente',
        'columna_13': 'Fecha_Archivo',
        'columna_14': 'Nombre_Archivo',
    },
    'CRB': {
        'columna_01': 'Número_del_deudor',
        'columna_02': 'Número_de_recibo',
        'columna_03': 'Fecha_de_recibo',
        'columna_04': 'Monto_del_recibo',
        'columna_05': 'Moneda',
        'columna_06': 'Banco',
        'columna_07': 'Cuenta_bancaria',
        'columna_08': 'Referencia_bancaria',
        'columna_09': 'Status',
        'columna_10': 'Fecha_de_reversión',
        'columna_11': 'Código_de_reversión',
        'columna_12': 'Comentario_a_la_reversión',
        'columna_13': 'Fideicomitente',
        'columna_14': 'Fecha_Archivo',
        'columna_15': 'Nombre_Archivo',
    },
    'NCR': {
        'columna_01': 'Numero de contrato',
        'columna_02': 'Tipo',
        'columna_03': 'Monto',
        'columna_04': 'Fecha Aplicación',
        'columna_05': 'Tipo Aplicación',
        'columna_06': 'Codigo Aplicación',
        'columna_07': 'Fecha_Archivo',
        'columna_08': 'Nombre_Archivo',
    },
    'DELETION_COB': {
        'columna_01': 'Número_de_recibo',
        'columna_02': 'Número_de_deudor',
        'columna_03': 'Número_de_contrato',
        'columna_08': 'Concepto_a_ceder',
        'columna_09': 'Fecha_de_depósito',
        'columna_10': 'Fecha_de_aplicación',
        'columna_11': 'Monto_de_distribución',
        'columna_12': 'Fideicomisario',
        'columna_13': 'Factura_de_Cliente',
        'columna_14': 'Pago',
        'columna_15': 'Contrato_Credito',
        'columna_16': 'Fideicomitente',
        'columna_17': 'Fecha_Archivo',
        'columna_18': 'Nombre_Archivo',
    },
    'DELETION_CRB': {
        'columna_01': 'Número_del_deudor',
        'columna_02': 'Número_de_recibo',
        'columna_03': 'Fecha_de_recibo',
        'columna_04': 'Monto_del_recibo',
        'columna_05': 'Moneda',
        'columna_06': 'Banco',
        'columna_07': 'Cuenta_bancaria',
        'columna_08': 'Referencia_bancaria',
        'columna_09': 'Status',
        'columna_10': 'Fecha_de_reversión',
        'columna_11': 'Código_de_reversión',
        'columna_12': 'Comentario_a_la_reversión',
        'columna_13': 'Fideicomitente',
        'columna_14': 'Fecha_Archivo',
        'columna_15': 'Nombre_Archivo',
    },

    # ------------------------- CxC -------------------------------
    'APLICACIONES': {
        'columna_01': 'fecha_generacion',
        'columna_02': 'id_fideicomiso',
        'columna_03': 'alias',
        'columna_04': 'numero_recibo',
        'columna_05': 'monto',
        'columna_06': 'monto_fondeado',
        'columna_07': 'fecha_distribucion',
        'columna_08': 'clave_deudor',
        'columna_09': 'nombre_deudor',
        'columna_10': 'esta_conciliada',
        'columna_11': 'fecha_deposito',
        'columna_12': 'no_cuenta',
        'columna_13': 'abreviacion',
        'columna_14': 'id_aplicacion',
        'columna_15': 'monto_transferencia',
        'columna_16': 'numero_contrato',
        'columna_17': 'concepto_archivo',
        'columna_18': 'fideicomisario',
        'columna_19': 'identificador_cliente',
        'columna_20': 'fecha_archivo',
        'columna_21': 'fecha_reversion',
        'columna_22': 'nombre_archivo',
        'columna_23': 'fideicomitente'
    },
    'RECIBOS': {
        'columna_01': 'fecha_generacion',
        'columna_02': 'id_fideicomiso',
        'columna_03': 'alias',
        'columna_04': 'banco',
        'columna_05': 'numero_cuenta',
        'columna_06': 'numero_recibo',
        'columna_07': 'fecha_deposito',
        'columna_08': 'monto',
        'columna_09': 'referencia_bancaria',
        'columna_10': 'conciliado',
        'columna_11': 'saldo_restante',
        'columna_12': 'id_recibo',
        'columna_13': 'clave_deudor',
        'columna_14': 'nombre_deudor',
        'columna_15': 'nombre_archivo',
        'columna_16': 'fideicomitente'
    },
    'DEPOSITOS': {
        'columna_01': 'fecha_generacion',
        'columna_02': 'id_fideicomiso',
        'columna_03': 'alias',
        'columna_04': 'banco',
        'columna_05': 'numero_cuenta',
        'columna_06': 'categoria',
        'columna_07': 'fecha_deposito',
        'columna_08': 'monto_deposito',
        'columna_09': 'referencia',
        'columna_10': 'concepto',
        'columna_11': 'referencia_ampliada',
        'columna_12': 'saldo_restante',
        'columna_13': 'esta_conciliado',
        'columna_14': 'id_recibo',
    },
    'TRANSFERENCIAS': {
        'columna_01': 'fecha_generacion',
        'columna_02': 'id_deposito',
        'columna_03': 'id_recibo',
        'columna_04': 'monto_transferencia',
        'columna_05': 'fecha_transferencia',
        'columna_06': 'id_tipo_transferencia',
        'columna_07': 'nombre_regla',
        'columna_08': 'descripcion',
    },

    # ------------------------- Errores ----------------------------
    'ERRORES_CDD': {
        'columna_01': 'ID',
        'columna_02': 'Key',
        'columna_03': 'Value',
        'columna_04': 'line',
        'columna_05': 'Id_nom',
        'columna_06': 'Nombre',
        'columna_07': 'Recibo',
        'columna_08': 'Fecha',
        'columna_09': 'Fidecomisario',
        'columna_10': 'Fidecomitente',
        'columna_11': 'Fecha_Archivo',
        'columna_12': 'Nombre_Archivo',
    },
    'ERRORES_COB': {
        'columna_01': 'ID',
        'columna_02': 'Key',
        'columna_03': 'Value',
        'columna_04': 'line',
        'columna_05': 'Numero_Recibos',
        'columna_06': 'Clave_Deudor',
        'columna_07': 'Contrato',
        'columna_08': 'Concepto_A_ceder',
        'columna_09': 'Fecha_Deposito',
        'columna_10': 'Fecha_Aplicacion',
        'columna_11': 'Monto_Distribuido',
        'columna_12': 'Fideicomiso',
        'columna_13': 'Fecha_Archivo',
        'columna_14': 'Nombre_Archivo',
    },
    'ERRORES_CRB': {
        'columna_01': 'ID',
        'columna_02': 'Key',
        'columna_03': 'Value',
        'columna_04': 'line',
        'columna_05': 'Deudor',
        'columna_06': 'Numero_Recibo',
        'columna_07': 'Fecha_Deposito',
        'columna_08': 'Monto',
        'columna_09': 'Moneda',
        'columna_10': 'Fideicomiso',
        'columna_11': 'Cuenta',
        'columna_12': 'Referencia',
        'columna_13': 'Fideicomitente',
        'columna_14': 'Fecha_Archivo',
        'columna_15': 'Nombre_Archivo',
    },
    'ERRORES_DELETIONCOB': {
        'columna_01': 'ID',
        'columna_02': 'Key',
        'columna_03': 'Value',
        'columna_04': 'line',
        'columna_05': 'Numero_Recibos',
        'columna_06': 'Clave_Deudor',
        'columna_07': 'Contrato',
        'columna_08': 'Concepto_A_ceder',
        'columna_09': 'Fecha_Deposito',
        'columna_10': 'Fecha_Aplicacion',
        'columna_11': 'Monto_Distribuido',
        'columna_12': 'Fideicomiso',
        'columna_13': 'Fecha_Archivo',
        'columna_14': 'Nombre_Archivo',
    },
    'ERRORES_NCR': {
        'columna_01': 'id',
        'columna_02': 'numero_contrato',
        'columna_03': 'tipo_liberacion',
        'columna_04': 'monto',
        'columna_05': 'fecha_aplicacion',
        'columna_06': 'tipo_aplicacion',
        'columna_07': 'codigo_aplicacion',
        'columna_08': 'id_tiempo',
        'columna_09': 'id_archivo',
        'columna_10': 'created_at',
        'columna_11': 'id_estatus',
        'columna_12': 'numero_linea_archivo',
        'columna_13': 'estatus',
        'columna_14': 'Fecha_Archivo',
        'columna_15': 'Nombre_Archivo',
    },
}

# ========================== normalización / fechas ==========================
def _strip_accents(s: str) -> str:
    if s is None: return ""
    nfkd = unicodedata.normalize("NFKD", s)
    return "".join([c for c in nfkd if not unicodedata.combining(c)])

def _norm_colname(name: str) -> str:
    name = _strip_accents(str(name)).lower()
    name = re.sub(r'\s+', '_', name)
    name = re.sub(r'[^a-z0-9_]', '', name)
    return name

def _norm_value_series(s: pd.Series, strip_leading_zeros: bool = False) -> pd.Series:
    out = s.astype(str).str.strip()
    if strip_leading_zeros:
        out = out.str.replace(r'^0+', '', regex=True)
    return out

_EXCEL_EPOCH = "1899-12-30"

def is_date_col(col_name: str) -> bool:
    col_l = str(col_name).lower()
    return ('fecha' in col_l) or ('date' in col_l)

def to_date_series(s: pd.Series) -> pd.Series:
    if pd.api.types.is_datetime64_any_dtype(s) or pd.api.types.is_datetime64tz_dtype(s):
        return s.dt.date
    if not s.dropna().empty and all(hasattr(x,"year") and hasattr(x,"month") and hasattr(x,"day") for x in s.dropna().head(50)):
        return pd.to_datetime(s, errors='coerce').dt.date
    s_str = s.astype(str).str.strip().replace({'': pd.NA, 'nan': pd.NA, 'NaN': pd.NA})
    uniques = s_str.dropna().unique()
    cache = {}
    parsed_text = pd.to_datetime(pd.Series(uniques), errors='coerce', dayfirst=True, infer_datetime_format=True)
    for k,v in zip(uniques, parsed_text):
        if pd.notna(v): cache[k]=v
    digits = pd.Series(uniques).str.replace(r'[^0-9]', '', regex=True)
    ymd_ok = pd.to_datetime(digits.where(digits.str.len()==8), format="%Y%m%d", errors='coerce')
    for k,v in zip(uniques, ymd_ok):
        if pd.notna(v) and k not in cache: cache[k]=v
    s_num_map = pd.to_numeric(pd.Series(uniques), errors='coerce')
    xls_ok = pd.to_datetime(s_num_map, unit='D', origin=_EXCEL_EPOCH, errors='coerce')
    for k,v in zip(uniques, xls_ok):
        if pd.notna(v) and k not in cache: cache[k]=v
    out = s_str.map(cache).astype('datetime64[ns]')
    return out.dt.date

# ===================== conjuntos y separadores =====================
CXC_SET = {'APLICACIONES', 'RECIBOS', 'DEPOSITOS', 'TRANSFERENCIAS'}
COLUMNAS_SIN_CEROS = {
    'CDD': {'Número_del_deudor', 'Número_de_Contrato'},
    'COB': {'Número_de_recibo', 'Número_de_deudor', 'Número_de_contrato'},
    'CRB': {'Número_del_deudor', 'Número_de_recibo'},
    'NCR': set(),
    'DELETION_COB': {'Número_de_recibo', 'Número_de_deudor', 'Número_de_contrato'},
    'DELETION_CRB': {'Número_del_deudor', 'Número_de_recibo'},
    'APLICACIONES': {'numero_recibo', 'numero_contrato', 'id_aplicacion'},
    'RECIBOS': {'numero_recibo', 'id_recibo'},
    'DEPOSITOS': {'id_deposito', 'numero_cuenta'},
    'TRANSFERENCIAS': {'id_deposito', 'id_recibo'},
    'ERRORES_CDD': set(), 'ERRORES_COB': set(), 'ERRORES_CRB': set(), 'ERRORES_DELETIONCOB': set(), 'ERRORES_NCR': set(),
}

# ============================== Lector unificado ==============================
def smart_read_table(ruta: str, columnas: list[str], tipo: str | None = None, verbose: bool = False) -> pd.DataFrame:

    # ---------- CxC: camino turbo ----------
    if tipo in CXC_SET:
        if _PYARROW_OK:
            try:
                df = pd.read_csv(
                    ruta, sep='|', names=columnas, header=0,
                    dtype='string[pyarrow]', engine='pyarrow',
                    memory_map=True, na_filter=False
                )
                if verbose: print(f" CxC (pyarrow): {len(df)} filas")
                return df
            except Exception:
                pass
        try:
            df = pd.read_csv(
                ruta, sep='|', names=columnas, header=0,
                dtype=str, engine='c', memory_map=True,
                quoting=csv.QUOTE_NONE, low_memory=False
            )
            if verbose: print(f" CxC (engine C): {len(df)} filas")
            return df
        except Exception:
            df = pd.read_csv(
                ruta, sep='|', names=columnas, header=0,
                dtype=str, engine='python', on_bad_lines='skip'
            )
            if verbose: print(f" CxC (python tolerante): {len(df)} filas")
            return df

    # ---------- Administración / Errores ----------
    seps = ['|', ',', '\t', ';']
    # 1) engine C con header=0 (saltamos encabezado real) y luego header=None
    for sep_try in seps:
        try:
            df = pd.read_csv(
                ruta, sep=sep_try, names=columnas,
                header=0, dtype=str, engine='c',
                memory_map=True, low_memory=False
            )
            if df.shape[1] == len(columnas):
                if verbose: print(f"✅ Admin/Errores C-engine (skip header) sep={repr(sep_try)} filas={len(df)}")
                return df
        except Exception:
            pass
        try:
            df = pd.read_csv(
                ruta, sep=sep_try, names=columnas,
                header=None, dtype=str, engine='c',
                memory_map=True, low_memory=False
            )
            if df.shape[1] == len(columnas):
                if verbose: print(f"✅ Admin/Errores C-engine (sin header) sep={repr(sep_try)} filas={len(df)}")
                return df
        except Exception:
            continue

    # 2) engine python tolerante
    for sep_try in seps:
        try:
            df = pd.read_csv(
                ruta, sep=sep_try, names=columnas,
                header=0, dtype=str, engine='python',
                on_bad_lines='skip'
            )
            if df.shape[1] == len(columnas):
                if verbose: print(f"⚠️ Admin/Errores python (skip header) sep={repr(sep_try)} filas={len(df)}")
                return df
        except Exception:
            pass
        try:
            df = pd.read_csv(
                ruta, sep=sep_try, names=columnas,
                header=None, dtype=str, engine='python',
                on_bad_lines='skip'
            )
            if df.shape[1] == len(columnas):
                if verbose: print(f"⚠️ Admin/Errores python (sin header) sep={repr(sep_try)} filas={len(df)}")
                return df
        except Exception:
            continue

    # Ultimo recurso para descartar
    return pd.read_csv(
        ruta, sep=',', names=columnas,
        header=0, dtype=str, engine='python',
        on_bad_lines='skip'
    )

# ============================== Buscador de archivo ===========================
def encontrar_archivo_mas_reciente(tipo: str, lookback_dias: int = 10, nombre_exacto: str | None = None) -> tuple[str, pd.DataFrame]:
    cfg = TIPOS[tipo]
    hoy = datetime.now()
    errores: list[tuple[str, str]] = []
    encontrados = 0

    # Nombre exacto
    if nombre_exacto:
        candidatos = []
        for i in range(0, lookback_dias + 1):
            fecha = hoy - timedelta(days=i)
            dir_dia = construir_ruta_dia(cfg['carpeta'], fecha)
            ruta = os.path.join(dir_dia, nombre_exacto)
            if os.path.isfile(ruta):
                candidatos.append(ruta)
        if candidatos:
            ruta = max(candidatos, key=os.path.getmtime)
            try:
                df = smart_read_table(ruta, columnas=cfg['columnas'], tipo=tipo)
                print(f"📂 Usando archivo (exacto): {os.path.basename(ruta)}")
                return ruta, df
            except Exception as e:
                raise RuntimeError(f"No se pudo leer el archivo exacto: {e}")
        else:
            print(f"⚠️ No se encontró el nombre exacto '{nombre_exacto}'. Sigo con búsqueda por patrón...")

    # Búsqueda por patrón (ultimo N días)
    for i in range(lookback_dias + 1):
        fecha = hoy - timedelta(days=i)
        dir_dia = construir_ruta_dia(cfg['carpeta'], fecha)
        if not os.path.isdir(dir_dia):
            continue
        base_name = nombre_objetivo(cfg['documento'], fecha)
        try:
            with os.scandir(dir_dia) as it:
                archivos = [e.name for e in it if e.is_file() and base_name in e.name]
            if not archivos:
                continue
            #preferir .txt en archivos
            archivos_txt = [n for n in archivos if n.lower().endswith(".txt")]
            archivos_otros = [n for n in archivos if n not in archivos_txt]

            for nombre_archivo in archivos_txt + archivos_otros:
                encontrados += 1
                ruta = os.path.join(dir_dia, nombre_archivo)
                try:
                    df = smart_read_table(ruta, columnas=cfg['columnas'], tipo=tipo)
                    print(f"📂 Usando archivo base: {nombre_archivo}")
                    return ruta, df
                except Exception as e:
                    errores.append((ruta, str(e)))
                    continue
        except Exception as e:
            errores.append((dir_dia, f"Listar: {e}"))
            continue

    if encontrados > 0 and errores:
        raise RuntimeError(
            f"Se encontraron {encontrados} archivo(s) con patrón pero ninguno se pudo leer.\n"
            + "\n".join([f"- {os.path.basename(r)} -> {err}" for r, err in errores[:8]])
        )
    raise FileNotFoundError(f"No se encontró {cfg['documento']}_YYYYMMDD en los últimos {lookback_dias} días en {cfg['carpeta']}")

# ================================ Utilidades filtro ===========================
def _limpia_texto_fecha_series(s: pd.Series) -> pd.Series:
    """Elimina invisibles, NBSP, recorta a 10, strip."""
    return (s.astype(str)
            .str.replace('\u200b','',regex=False)  # ZWSP
            .str.replace('\xa0',' ',regex=False)   # NBSP -> espacio
            .str.strip()
            .str.slice(0,10))

def _parse_iso_yyyy_mm_dd_to_date(s: pd.Series) -> pd.Series:
    """ ISO YYYY-MM-DD => datetime64[ns] => date"""
    return pd.to_datetime(s, format='%Y-%m-%d', errors='coerce').dt.date

# ================================ Filtrados ===================================
def filtrado_masivo(tipo: str, df_base: pd.DataFrame) -> pd.DataFrame:
    ejecutor = input("¿Quién ejecuta? (JC, ED, MD, MS, IT, KT, ST): ").strip().upper()
    ruta_excel = rutas_usuarios.get(ejecutor)
    if not ruta_excel or not os.path.exists(ruta_excel):
        raise FileNotFoundError(f"No existe el Excel del usuario: {ruta_excel}")

    mapeo = USUARIO_A_DESTINO.get(tipo, {})

# ---------- Lee TODAS las hojas y auto-selecciona la mejor por columnas mapeadas ----------
    hojas = pd.read_excel(ruta_excel, dtype=str, sheet_name=None)
    mejor_df = None
    mejor_score = -1
    mejor_nombre = None

    for nombre_hoja, df_tmp in hojas.items():
        # normaliza encabezados
        df_tmp.columns = [_norm_colname(c) for c in df_tmp.columns]
        # renombrado automático: si el usuario puso el NOMBRE DESTINO en vez del nombre de usuario (columna_xx)
        for col_usr_key, col_dest in mapeo.items():
            col_usr_norm = _norm_colname(col_usr_key)
            dest_norm = _norm_colname(col_dest)
            # Si no existe la 'columna_xx' pero sí el nombre destino,se renombralo a 'columna_xx'
            if col_usr_norm not in df_tmp.columns and dest_norm in df_tmp.columns:
                df_tmp = df_tmp.rename(columns={dest_norm: col_usr_norm})
        # score = suma de no-nulos en TODAS las columnas mapeadas que existan
        score = 0
        for col_usr_key in mapeo.keys():
            col_usr_norm = _norm_colname(col_usr_key)
            if col_usr_norm in df_tmp.columns:
                score += df_tmp[col_usr_norm].notna().sum()

        if score > mejor_score:
            mejor_score = score
            mejor_df = df_tmp
            mejor_nombre = nombre_hoja

    if mejor_df is None:
        nombre_primera = next(iter(hojas.keys()))
        mejor_df = hojas[nombre_primera]
        mejor_df.columns = [_norm_colname(c) for c in mejor_df.columns]
        mejor_nombre = nombre_primera

    df_usr = mejor_df
    #print(f"[INFO] Hoja seleccionada: '{mejor_nombre}' con score={mejor_score}")

    # ------------------- Diagnóstico por columna mapeada -------------------
    for col_usr_key in mapeo.keys():
        col_usr_norm = _norm_colname(col_usr_key)
        nn = df_usr[col_usr_norm].notna().sum() if col_usr_norm in df_usr.columns else 0
        #print(f"[CHK] {col_usr_norm}: existe={col_usr_norm in df_usr.columns}, no-nulos={nn}")

    # ------------------- Construcción de criterios -------------------
    criterios: list[tuple[str, set, bool]] = []
    cols_sin_zeros = COLUMNAS_SIN_CEROS.get(tipo, set())

    for col_usr_key, col_dest in mapeo.items():
        col_usr_norm = _norm_colname(col_usr_key)
        if col_usr_norm not in df_usr.columns or col_dest not in df_base.columns:
            continue
        serie_vals = df_usr[col_usr_norm].dropna()
        if serie_vals.empty:
            continue

        if is_date_col(col_dest):
            # Limpieza + ISO estricto para valores del Excel
            serie_iso = _limpia_texto_fecha_series(serie_vals)
            vals_dt = _parse_iso_yyyy_mm_dd_to_date(serie_iso)
            valores = set([v for v in vals_dt.dropna().tolist()])
            if valores:
                criterios.append((col_dest, valores, False))
        else:
            strip0 = col_dest in cols_sin_zeros
            valores = set(
                _norm_value_series(serie_vals, strip_leading_zeros=strip0)
                .replace({'': pd.NA}).dropna().tolist()
            )
            if valores:
                criterios.append((col_dest, valores, strip0))

    if not criterios:
        print("No se encontraron criterios validos en el Excel del usuario para este tipo.")
        return df_base.iloc[0:0].copy()
    # Normaliza SOLO columnas a usar
    cols_objetivo = [c for (c, _, _) in criterios if c in df_base.columns]
    df_work = df_base[cols_objetivo].copy()

    # fechas (limpieza equivalente al archivo base)
    for c, _, _ in criterios:
        if c in df_work.columns and is_date_col(c):
            s = _limpia_texto_fecha_series(df_work[c])
            try:
                df_work[c] = _parse_iso_yyyy_mm_dd_to_date(s)
            except Exception:
                df_work[c] = to_date_series(df_work[c])
    # texto/ids
    for c, _, strip0 in criterios:
        if c in df_work.columns and not is_date_col(c):
            df_work[c] = _norm_value_series(df_work[c], strip_leading_zeros=strip0)

    mask_total = pd.Series(False, index=df_base.index)
    for col_dest, valores, _ in criterios:
        if col_dest not in df_work.columns or not valores:
            continue
        mask_total |= df_work[col_dest].isin(valores)

    df_filtrado = df_base.loc[mask_total].copy()

    # ---------- Diagnóstico útil (antes del return) ----------
    #print(f"[DBG] Criterios aplicados: {len(criterios)}")
    for col_dest, valores, _ in criterios:
        ejemplo = sorted(list(valores))[:10]
        print(f"[DBG] - {col_dest}: {len(valores)} valores (ej: {ejemplo})")

    # Si entre los criterios hay fechas y existen en base, valida faltantes por cada columna fecha
    for col_dest, valores, _ in criterios:
        if is_date_col(col_dest) and col_dest in df_base.columns:
            try:
                base_fechas = _parse_iso_yyyy_mm_dd_to_date(_limpia_texto_fecha_series(df_base[col_dest])).dropna()
                faltantes = [f for f in sorted(list(valores)) if (base_fechas == f).sum() == 0]
                if faltantes:
                    print(f"[DBG] Fechas de {col_dest} inexistentes en base (muestra): {faltantes[:10]} ...")
            except Exception as _e:
                print(f"[DBG] No pude diagnosticar fechas en base para {col_dest}: {_e}")

    print(f"🎯 Filtrado masivo -> {len(df_filtrado)} registros (de {len(df_base)})")
    return df_filtrado

def filtrado_individual(tipo: str, df_base: pd.DataFrame) -> pd.DataFrame:
    cols = list(df_base.columns)
    print("\nColumnas disponibles:")
    for c in cols:
        print(f" - {c}")

    seleccionadas: list[str] = []
    while True:
        col = input("¿Qué columna deseas filtrar? (pon 'ALTO' para parar): ").strip()
        if col.lower() == 'alto':
            break
        if col not in df_base.columns:
            print("Columna no válida. Intenta de nuevo.")
            continue
        if col not in seleccionadas:
            seleccionadas.append(col)

    if not seleccionadas:
        print("No se seleccionaron columnas. Se regresa vacío.")
        return df_base.iloc[0:0].copy()

    filtros = {}
    for col in seleccionadas:
        valores = input(f"¿Qué valores deseas filtrar para '{col}'? (separa por comas): ").strip()
        lista = [v.strip() for v in valores.split(',') if v.strip() != '']
        filtros[col] = set(lista)

    # Prepara df normalizado SOLO para columnas seleccionadas
    df_work = df_base[seleccionadas].copy()

    # Identifica columnas de fecha
    sel_date_cols = [c for c in seleccionadas if is_date_col(c)]
    for c in sel_date_cols:
        try:
            s = _limpia_texto_fecha_series(df_work[c])
            df_work[c] = _parse_iso_yyyy_mm_dd_to_date(s)
        except Exception:
            df_work[c] = to_date_series(df_work[c])

    for c in seleccionadas:
        if c not in sel_date_cols:
            df_work[c] = df_work[c].astype(str).str.strip()

    # Construir máscara (AND entre columnas)
    mask_total = pd.Series(True, index=df_base.index)
    for col, valores in filtros.items():
        if col in sel_date_cols:
            vals_series = pd.Series(list(valores))
            vals_dt = _parse_iso_yyyy_mm_dd_to_date(_limpia_texto_fecha_series(vals_series))
            vals_set = set([v for v in vals_dt.dropna().tolist()])
            mask_col = df_work[col].isin(vals_set)
        else:
            vals_norm = set([str(v).strip() for v in valores])
            mask_col = df_work[col].isin(vals_norm)
        mask_total &= mask_col

    df_filtrado = df_base.loc[mask_total].copy()
    print(f"🎯 El filtrado resultó en {len(df_filtrado)} registros.")
    return df_filtrado

# ================================ Guardado ===================================
def guardar_extracto(df_filtrado: pd.DataFrame, tag: str="COB") -> None:
    if df_filtrado is None or df_filtrado.empty:
        print("No hay datos para guardar.")
        return

    guardar = input("¿Deseas guardar este extracto? (si/no): ").strip().lower()
    if guardar != 'si':
        print("No se guardó el archivo.")
        return

    formato = input("¿Formato? (csv/txt/xlsx) [csv]: ").strip().lower() or "csv"
    if formato not in ['xlsx', 'txt', 'csv']:
        print("❌ Formato no válido. Solo 'xlsx', 'txt' o 'csv'.")
        return

    nombre_archivo = f"Extracto_{tag}_{hoy_yyyymmdd()}.{formato}"

    try:
        if formato == "xlsx":
            try:
                df_filtrado.to_excel(nombre_archivo, index=False, engine="xlsxwriter")
            except Exception:
                df_filtrado.to_excel(nombre_archivo, index=False)
        else:
            # CxC => pipe; Admin/Errores => coma
            sep = '|' if (tag in CXC_SET) else ','
            df_filtrado.to_csv(nombre_archivo, index=False, sep=sep, encoding="utf-8-sig")

        print(f"✅ Archivo generado: {nombre_archivo}")
        if _COLAB:
            files.download(nombre_archivo)
        else:
            print("📁 Ubicación:", os.path.abspath(nombre_archivo))
    except Exception as e:
        print(f"❌ Error al guardar el archivo: {e}")

# =============================== Menú principal ===============================
MENU_1 = {
    '1': ('Archivos de administración', ['CDD', 'CRB', 'COB', 'NCR', 'DELETION_CRB', 'DELETION_COB']),
    '2': ('CXC', ['APLICACIONES', 'DEPOSITOS', 'RECIBOS', 'TRANSFERENCIAS']),
    '3': ('Errores', ['ERRORES_CDD', 'ERRORES_COB', 'ERRORES_CRB', 'ERRORES_DELETIONCOB', 'ERRORES_NCR'])
}

def seleccionar_categoria_y_tipo() -> str:
    print("¿Con qué archivo deseas trabajar?")
    for key, (titulo, _) in MENU_1.items():
        print(f"  {key}) {titulo}")
    op = input("Elige una opción (1, 2, 3): ").strip()
    if op not in MENU_1:
        raise ValueError("Opción no válida")

    titulo, opciones = MENU_1[op]
    print(f"\n¿Qué {titulo.split()[0]}?")
    for i, t in enumerate(opciones, 1):
        print(f"  {i}) {t}")
    sub = input("Elige una opción (número): ").strip()
    try:
        idx = int(sub) - 1
        tipo = opciones[idx]
    except Exception:
        raise ValueError("Opción de submenú no válida")

    return tipo

# ================================== Main =====================================
def main() -> None:
    try:
        tipo = seleccionar_categoria_y_tipo()
        if tipo not in TIPOS:
            raise KeyError(f"Tipo desconocido: {tipo}")

        # 1) Carga archivo base (últimos N días)
        ruta, df_base = encontrar_archivo_mas_reciente(tipo, lookback_dias=10)

        # 2) Elegir tipo de filtrado
        modo = input("¿Es un filtrado masivo o individual? ").strip().lower()
        if modo not in ['masivo', 'individual']:
            raise ValueError("Debes indicar 'masivo' o 'individual'")

        if modo == 'masivo':
            df_filtrado = filtrado_masivo(tipo, df_base)
        else:
            df_filtrado = filtrado_individual(tipo, df_base)

        # 3) Guardar
        guardar_extracto(df_filtrado, tag=tipo)

    except Exception as e:
        print(f"❌ Error: {e}")

if __name__ == "__main__":
    main()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
¿Con qué archivo deseas trabajar?
  1) Archivos de administración
  2) CXC
  3) Errores


KeyboardInterrupt: Interrupted by user